<a href="https://colab.research.google.com/github/kim-bo-saeng/dacon_jeju/blob/master/jeju_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
submission = pd.read_csv('/gdrive/My Drive/data_analysis/submission.csv')
print(submission.shape)
submission.tail()

(1394, 5)


,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0
1390,1390,202007,충북,한식 음식점업,0
1391,1391,202007,충북,호텔업,0
1392,1392,202007,충북,화장품 및 방향제 소매업,0
1393,1393,202007,충북,휴양콘도 운영업,0


In [ ]:
train = pd.read_csv('/gdrive/My Drive/data_analysis/201901-202003.csv')
print(train.shape)
train.tail()

(24697792, 12)


,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6
24697791,202003,충북,충주시,휴양콘도 운영업,충북,충주시,60s,1,5,3,194000,3


In [ ]:
train.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [ ]:
train.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

In [ ]:
mini_train = train[["REG_YYMM", "CARD_SIDO_NM", "STD_CLSS_NM", "AMT"]]
mini_train.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,311200
1,201901,강원,건강보조식품 소매업,1374500
2,201901,강원,건강보조식품 소매업,818700
3,201901,강원,건강보조식품 소매업,1717000
4,201901,강원,건강보조식품 소매업,1047300


In [ ]:
pivot_mini_train = pd.pivot_table(data = mini_train, index = ["STD_CLSS_NM","CARD_SIDO_NM", "REG_YYMM"], fill_value = "AMT", aggfunc = "sum")
pivot_mini_train.head()

AMT
STD_CLSS_NM CARD_SIDO_NM REG_YYMM           
건강보조식품 소매업  강원           201901    148354746
                         201902    146715871
                         201903    109439025
                         201904    106548473
                         201905    148115738

In [ ]:
pivot_mini_train.reset_index(inplace = True)
pivot_mini_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9438 entries, 0 to 9437
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STD_CLSS_NM   9438 non-null   object
 1   CARD_SIDO_NM  9438 non-null   object
 2   REG_YYMM      9438 non-null   int64 
 3   AMT           9438 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 295.1+ KB


In [ ]:
def change_mm(data):
  if data > 201913:
    return data - 202000
  else:
    return data - 201900

pivot_mini_train["REG_MM"] = pivot_mini_train["REG_YYMM"].apply(change_mm)
pivot_mini_train[["REG_YYMM", "REG_MM"]]

,REG_YYMM,REG_MM
0,201901,1
1,201902,2
2,201903,3
3,201904,4
4,201905,5
...,...,...
9433,201911,11
9434,201912,12
9435,202001,1
9436,202002,2


In [ ]:
pivot_mini_train

,STD_CLSS_NM,CARD_SIDO_NM,REG_YYMM,AMT,REG_MM
0,건강보조식품 소매업,강원,201901,148354746,1
1,건강보조식품 소매업,강원,201902,146715871,2
2,건강보조식품 소매업,강원,201903,109439025,3
3,건강보조식품 소매업,강원,201904,106548473,4
4,건강보조식품 소매업,강원,201905,148115738,5
...,...,...,...,...,...
9433,휴양콘도 운영업,충북,201911,124117555,11
9434,휴양콘도 운영업,충북,201912,139568962,12
9435,휴양콘도 운영업,충북,202001,222073112,1
9436,휴양콘도 운영업,충북,202002,47085830,2


In [ ]:
import statsmodels.api as sm


model = sm.OLS.from_formula("AMT ~ C(REG_MM):C(STD_CLSS_NM):C(CARD_SIDO_NM)", pivot_mini_train)
result = model.fit()
result.summary()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
sub = submission.copy()

sub["REG_MM"] = sub["REG_YYMM"].apply(change_mm)
sub[["REG_YYMM", "REG_MM"]]
sub.tail()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,REG_MM
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0,7
1390,1390,202007,충북,한식 음식점업,0,7
1391,1391,202007,충북,호텔업,0,7
1392,1392,202007,충북,화장품 및 방향제 소매업,0,7
1393,1393,202007,충북,휴양콘도 운영업,0,7
